Compact pathway creator function

Defines all pathways and returns a model with the selected pathway added.
The function can be called alone or get assigned to a new variable, but it always alters the handed model. In order to not touch manipulate the existing model work within with model: or create a copy of your model with model.copy() before handing that copy to this function.

Function

create_pathway(model,path_selector)

model ... needs to be handed a model, since it doesnt import it itself to save time when function if called multiple times
path_selector ... determines which path is added
Import the function into your script with the following code, thic function script has to be in the same folder.

%run -i create_pathway.ipynb

Pathway legend

0 -> no additional reactions
1 -> beta alanine pathway
2 -> transferase malonyl-CoA pathway
3 -> oxidoreductive malonyl-CoA pathway
4 -> lactate pathway


In [ ]:
path_name = ['no additional reactions','route1', 'route2' ]